In [5]:
%matplotlib inline

In [47]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
from skimage.io import imread
import scipy.ndimage
from scipy import stats
from scipy.stats import ttest_ind, levene, shapiro
from scipy.stats import ttest_rel
from scipy.stats import f_oneway
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Hypothesis Testing Exercise
## Comparing Distributions. Testing statistical hypotheses. p-value. Goodness of fit

### Problem 1. t-test
You're given a dataset which describes the age at death among members of the sovereignty, aristocracy, and gentry. There is an explanation [here](http://www.stat.ufl.edu/~winner/data/agedeath.txt) and the actual data is [here](http://www.stat.ufl.edu/~winner/data/agedeath.dat). Get to know the data; plot histograms and / or boxplots, or other graphs as needed. Note that all ages are only for a certain social class of people and they won't represent the entire population of people.

Do the mean ages differ significantly ($\alpha = 5\%$)? Perform a t-test. Do you need an independent or paired test?

In [48]:
column_names = ['Social_Class', 'Age_at_Death', 'Sequential_Number']
data = pd.read_csv("agedeath.dat", sep='\s+', names=column_names, header=None)
data = data[['Social_Class', 'Age_at_Death']]

print("Dataset overview:")
print("=" * 19)
print(f"Dataset (processed) shape: {data.shape}")
print("Data types:")
print(data.dtypes)
print("\nUnique social classes:")
print(data['Social_Class'].unique())
print("\nValue counts for social classes:")
print(data['Social_Class'].value_counts())

print("\nDescriptive statistics:")
print("=" * 75)
descriptive_stats = data.groupby('Social_Class')['Age_at_Death'].describe()
print(descriptive_stats.round(2))

class_counts = data['Social_Class'].value_counts()
top_classes = class_counts.head(2).index.tolist()

group1 = data[data['Social_Class'] == top_classes[0]]['Age_at_Death']
group2 = data[data['Social_Class'] == top_classes[1]]['Age_at_Death']
group1_name = f"Class {top_classes[0]}"
group2_name = f"Class {top_classes[1]}"

print(f"\nComparing the two largest groups:")
print(f"Group 1 ({group1_name}): {len(group1)} observations")
print(f"Group 2 ({group2_name}): {len(group2)} observations")
print(f"Group 1 mean: {group1.mean():.2f}")
print(f"Group 2 mean: {group2.mean():.2f}")
print()

plt.figure(figsize = (12, 10))

# Histogram for Gentry
plt.subplot(3, 3, 1)
gent_data = data[data['Social_Class'] == 'gent']['Age_at_Death']
plt.hist(gent_data, bins = 30, color='skyblue', alpha=0.7, edgecolor='black')
plt.title(f'Gentry - Age at death distribution\n(n={len(gent_data)})')
plt.xlabel('Age at death')
plt.ylabel('Frequency')
plt.grid(True, alpha = 0.3)

# Histogram for Aristocracy
plt.subplot(3, 3, 2)
aris_data = data[data['Social_Class'] == 'aris']['Age_at_Death']
plt.hist(aris_data, bins = 30, color='lightcoral', alpha = 0.7, edgecolor='black')
plt.title(f'Aristocracy - Age at death distribution\n(n={len(aris_data)})')
plt.xlabel('Age at death')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Combined histograms
plt.subplot(3, 3, 3)
plt.hist(group1, bins=30, alpha=0.7, label=group1_name, color='blue')
plt.hist(group2, bins=30, alpha=0.7, label=group2_name, color='red')
plt.xlabel('Age at Death')
plt.ylabel('Frequency')
plt.title('Histogram of Age at death by Social class')
plt.legend()
plt.grid(True, alpha=0.3)

# Box plots
plt.subplot(3, 3, 4)
data.boxplot(column='Age_at_Death', by='Social_Class', ax=plt.gca())
plt.title('Box plot of Age at death by Social class')
plt.suptitle('')  # Remove default title
plt.xlabel('Social class')
plt.ylabel('Age at death')

# Q-Q plots for normality check
plt.subplot(3, 3, 5)
stats.probplot(group1, dist="norm", plot=plt)
plt.title(f'Q-Q Plot - {group1_name}')
plt.grid(True, alpha=0.3)

plt.subplot(3, 3, 6)
stats.probplot(group2, dist="norm", plot=plt)
plt.title(f'Q-Q Plot - {group2_name}')
plt.grid(True, alpha=0.3)

plt.subplot(3, 3, 7)
from scipy.stats import gaussian_kde
if len(group1) > 1:
    kde1 = gaussian_kde(group1)
    x1_range = np.linspace(group1.min(), group1.max(), 100)
    plt.plot(x1_range, kde1(x1_range), alpha=0.7, label=group1_name, color='blue', linewidth=2)

if len(group2) > 1:
    kde2 = gaussian_kde(group2)
    x2_range = np.linspace(group2.min(), group2.max(), 100)
    plt.plot(x2_range, kde2(x2_range), alpha=0.7, label=group2_name, color='red', linewidth=2)

plt.xlabel('Age at death')
plt.ylabel('Density')
plt.title('Density plot of Age at death')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nStatistical Tests and Assumptions:")
print("=" * 40)

print("1. Normality tests (Shapiro-Wilk):")
stat1, p1 = shapiro(group1)
stat2, p2 = shapiro(group2)
print(f"   Group 1: W = {stat1:.4f}, p-value = {p1:.4f}")
print(f"   Group 2: W = {stat2:.4f}, p-value = {p2:.4f}")

alpha = 0.05
if p1 > alpha and p2 > alpha:
    print("   Both groups appear to be normally distributed (p > 0.05)")
    normality_ok = True
else:
    print("   At least one group may not be normally distributed (p ≤ 0.05)")
    normality_ok = False

print("\n2. Equal variances test (Levene's test):")
stat_lev, p_lev = levene(group1, group2)
print(f"   Levene's test: W = {stat_lev:.4f}, p-value = {p_lev:.4f}")

if p_lev > alpha:
    print("   Equal variances assumption is satisfied (p > 0.05)")
    equal_var = True
else:
    print("   Equal variances assumption may be violated (p ≤ 0.05)")
    equal_var = False

# Determine appropriate t-test
print("\n3. T-Test selection:")
print("   Independent t-test because:")
print("   - We have two separate groups: different social classes")
print("   - The observations are NOT paired or matched")
print("   - Each individual appears ONLY ONCE in the dataset")

print("\nIndependent Samples T-Test Results:")
print("=" * 40)

if equal_var:
    t_stat, p_val = ttest_ind(group1, group2, equal_var = True)
    test_type = "Student's t-test (equal variances)"
else:
    t_stat, p_val = ttest_ind(group1, group2, equal_var = False)
    test_type = "Welch's t-test (unequal variances)"

print(f"Test used: {test_type}")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_val:.4f}")
print(f"Alpha level: {alpha}")

if equal_var:
    df = len(group1) + len(group2) - 2
else:
    # Welch's formula for unequal variances
    s1_sq = np.var(group1, ddof=1)
    s2_sq = np.var(group2, ddof=1)
    n1, n2 = len(group1), len(group2)
    df = (s1_sq/n1 + s2_sq/n2)**2 / ((s1_sq/n1)**2/(n1-1) + (s2_sq/n2)**2/(n2-1))

print(f"Degrees of freedom: {df:.1f}")

# Effect size (Cohen's d)
pooled_std = np.sqrt(((len(group1)-1)*np.var(group1, ddof=1) + (len(group2)-1)*np.var(group2, ddof=1)) / (len(group1)+len(group2)-2))
cohens_d = (np.mean(group1) - np.mean(group2)) / pooled_std
print(f"Cohen's d (effect size): {cohens_d:.4f}")

# Interpret effect size
if abs(cohens_d) < 0.2:
    effect_size = "small"
elif abs(cohens_d) < 0.5:
    effect_size = "small to medium"
elif abs(cohens_d) < 0.8:
    effect_size = "medium to large"
else:
    effect_size = "large"

print(f"Effect size interpretation: {effect_size}")

# Conclusion
print("\nConclusion:")
print("=" * 50)
if p_val < alpha:
    print(f"REJECT the null hypothesis (p = {p_val:.4f} < α = {alpha})")
    print("There IS a statistically significant difference between the mean ages at death")
    print(f"of the two social classes at the {alpha*100}% significance level.")
else:
    print(f"FAIL TO REJECT the null hypothesis (p = {p_val:.4f} ≥ α = {alpha})")
    print("There is NO statistically significant difference between the mean ages at death")
    print(f"of the two social classes at the {alpha*100}% significance level.")

print(f"\nMean difference: {np.mean(group1) - np.mean(group2):.2f} years")
print(f"95% Confidence interval for the difference:")

# Calculate 95% CI for the difference
from scipy.stats import t
se_diff = np.sqrt(np.var(group1, ddof=1)/len(group1) + np.var(group2, ddof=1)/len(group2))
t_critical = t.ppf(0.975, df)
diff = np.mean(group1) - np.mean(group2)
ci_lower = diff - t_critical * se_diff
ci_upper = diff + t_critical * se_diff
print(f"[{ci_lower:.2f}, {ci_upper:.2f}]")

Dataset overview:
Dataset (processed) shape: (6186, 2)
Data types:
Social_Class    object
Age_at_Death     int64
dtype: object

Unique social classes:
['aris' 'gent' 'sovr']

Value counts for social classes:
Social_Class
gent    2455
aris    2291
sovr    1440
Name: count, dtype: int64

Descriptive statistics:
               count   mean    std   min   25%   50%   75%    max
Social_Class                                                     
aris          2291.0  60.23  17.38  21.0  48.0  62.0  74.0  102.0
gent          2455.0  57.31  18.05  21.0  44.0  60.0  71.0  102.0
sovr          1440.0  54.23  16.07  21.0  43.0  56.0  66.0  101.0

Comparing the two largest groups:
Group 1 (Class gent): 2455 observations
Group 2 (Class aris): 2291 observations
Group 1 mean: 57.31
Group 2 mean: 60.23




Statistical Tests and Assumptions:
1. Normality tests (Shapiro-Wilk):
   Group 1: W = 0.9715, p-value = 0.0000
   Group 2: W = 0.9771, p-value = 0.0000
   At least one group may not be normally distributed (p ≤ 0.05)

2. Equal variances test (Levene's test):
   Levene's test: W = 4.0526, p-value = 0.0442
   Equal variances assumption may be violated (p ≤ 0.05)

3. T-Test selection:
   Independent t-test because:
   - We have two separate groups: different social classes
   - The observations are NOT paired or matched
   - Each individual appears ONLY ONCE in the dataset

Independent Samples T-Test Results:
Test used: Welch's t-test (unequal variances)
t-statistic: -5.6879
p-value: 0.0000
Alpha level: 0.05
Degrees of freedom: 4739.2
Cohen's d (effect size): -0.1650
Effect size interpretation: small

Conclusion:
REJECT the null hypothesis (p = 0.0000 < α = 0.05)
There IS a statistically significant difference between the mean ages at death
of the two social classes at the 5.0% significa

Let's try something else. In the `data/horse_beginners.dat` file, there's data about 8 people. Each person is a beginner in horse riding. They were asked to imagine how long their ride would take (for a fixed track length). After that, the actual times were measured.

Get acquainted with the data and draw some plots if you need (hint: boxplots are useful for comparing distributions). Are the people good at predicting their ride times? That is, are there significant differences between imagined and actual time (5%-level)? Also, is the imagined time **significantly longer** than the real time? Perform a t-test. Don't forget that the subjects are the same.

In [49]:
data = pd.read_csv("horse_beginners.dat", sep='\s+')

print("Horse Riding Time Analysis")
print("=" * 30)
print(data)

# Calculate differences
data['Difference'] = data['Imaginary'] - data['Actual']

print(f"\nDescriptive Statistics:")
print(f"Mean Imaginary: {data['Imaginary'].mean():.2f}")
print(f"Mean Actual: {data['Actual'].mean():.2f}")
print(f"Mean difference (Imaginary - Actual): {data['Difference'].mean():.2f}")
print()

plt.figure(figsize = (10, 6))

plt.subplot(1, 2, 1)
plt.boxplot([data['Imaginary'], data['Actual']], labels=['Imaginary', 'Actual'])
plt.title('Boxplot: Imaginary vs Actual times')
plt.ylabel('Time')

plt.subplot(1, 2, 2)
plt.plot(data['Imaginary'], data['Actual'], 'o', markersize=8)
min_val = min(data['Imaginary'].min(), data['Actual'].min())
max_val = max(data['Imaginary'].max(), data['Actual'].max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect prediction')
plt.xlabel('Imaginary time')
plt.ylabel('Actual time')
plt.title('Imaginary vs Actual times')
plt.legend()

plt.tight_layout()
plt.show()

shapiro_stat, shapiro_p = shapiro(data['Difference'])
print(f"\nNormality test (Shapiro-Wilk): p = {shapiro_p:.4f}")

print(f"Choose paired t-test: the subjects are the same.")
t_stat, p_val_two = ttest_rel(data['Imaginary'], data['Actual'])

print(f"\nPaired t-test results:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value (two-tailed): {p_val_two:.6f}")

alpha = 0.05
print(f"\nAre people good at predicting their ride times? (α = {alpha})")
if p_val_two < alpha:
    print("REJECT H0: There IS a significant difference")
    print("People are NOT good at predicting their ride times")
else:
    print("FAIL TO REJECT H0: No significant difference")
    print("We cannot say that people are not good in predicting their ride times")

print(f"\nIs imagined time significantly longer than actual?")
if data['Difference'].mean() > 0:
    p_val_one = p_val_two / 2
    print(f"p-value (one-tailed): {p_val_one:.6f}")
    if p_val_one < alpha:
        print("REJECT H0: Imagined time IS significantly longer than actual")
    else:
        print("FAIL TO REJECT H0: Imagined time is NOT significantly longer")
else:
    print("Mean difference is negative - people UNDERestimate, not overestimate their riding times.")
    print("Imagined time is actually significantly SHORTER than actual time.")
print()

Horse Riding Time Analysis
  Subject  Actual  Imaginary
1      S1   69.64      66.58
2      S2   62.26      25.59
3      S3   78.63      24.01
4      S4   76.00      38.35
5      S5   60.10      12.19
6      S6   68.51      34.25
7      S7   69.57       5.68
8      S8   74.48      15.02

Descriptive Statistics:
Mean Imaginary: 27.71
Mean Actual: 69.90
Mean difference (Imaginary - Actual): -42.19




Normality test (Shapiro-Wilk): p = 0.3148
Choose paired t-test: the subjects are the same.

Paired t-test results:
t-statistic: -6.1900
p-value (two-tailed): 0.000450

Are people good at predicting their ride times? (α = 0.05)
REJECT H0: There IS a significant difference
People are NOT good at predicting their ride times

Is imagined time significantly longer than actual?
Mean difference is negative - people UNDERestimate, not overestimate their riding times.
Imagined time is actually significantly SHORTER than actual time.



### Problem 2. ANOVA
Read [this](http://www.itl.nist.gov/div898/education/anova/newcar.dat) dataset. Get acquainted with it and plot graphics. Note that it's formatted a bit oddly. First, you have to skip some rows; second, the target variable is first (in most datasets, it's the last one). Well, that shouldn't bother you :).

Do the rates differ significantly ($\alpha = 0,05$) between cities? Apply one-way ANOVA to find out.

**Hint:** You have to group all values by city. You can get a dictionary of groups like this:
```python
groups = { k: v.tolist() for k, v in cars.groupby("City").Rate }
```

After that, you can use ```*groups.values()``` to pass all dictionary values as separate arguments to a function.

In [59]:
data = pd.read_csv("newcar.dat", sep='\s+', skiprows=25)
data.columns = ['Rate', 'City']

print("\nNew Car Interest Rates Dataset")
print("=" * 30)
print(f"\nDataset shape: {data.shape}")
print(f"Cities: {sorted(data['City'].unique())}")

print("\nDescriptive statistics by city:")
print(data.groupby('City')['Rate'].describe().round(3))
print()

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.boxplot([data[data['City'] == city]['Rate'] for city in sorted(data['City'].unique())],
            labels=[f'City {city}' for city in sorted(data['City'].unique())])
plt.title('Interest rates by city with outliers present')
plt.ylabel('Rate (%)')

plt.subplot(1, 2, 2)
city_means = data.groupby('City')['Rate'].mean()
plt.bar(range(1, 7), city_means.values)
plt.xlabel('City')
plt.ylabel('Mean rate (%)')
plt.title('Mean Interest rates by city')
plt.xticks(range(1, 7), [f'City {i}' for i in range(1, 7)])

plt.tight_layout()
plt.show()

print("\nAnova is robust to a small number of outliers.")
print("\nOne-way ANOVA results:")
print("=" * 22)

groups = {k: v.tolist() for k, v in data.groupby("City")["Rate"]}

f_stat, p_val = f_oneway(*groups.values())

print(f"F-statistic: {f_stat:.4f}")
print(f"p-value: {p_val:.6f}")

alpha = 0.05
print(f"Alpha: {alpha}")

if p_val < alpha:
    print(f"\nREJECT H0 (p < {alpha})")
    print("Interest rates DO differ significantly between cities.")
else:
    print(f"\nFAIL TO REJECT H0 (p >= {alpha})")
    print("Interest rates do NOT differ significantly between cities.")
print()


New Car Interest Rates Dataset

Dataset shape: (53, 2)
Cities: [1, 2, 3, 4, 5, 6]

Descriptive statistics by city:
      count    mean    std    min     25%    50%    75%    max
City                                                          
1       8.0  13.125  0.423  12.50  12.938  13.00  13.50  13.75
2       9.0  12.611  0.710  11.90  12.300  12.50  12.75  14.25
3       9.0  13.307  0.556  12.50  13.000  13.50  13.51  14.00
4       9.0  13.244  0.967  11.89  12.500  13.25  13.75  15.00
5       9.0  13.483  0.719  12.45  13.000  13.75  14.00  14.50
6       9.0  12.200  0.504  11.90  12.000  12.00  12.25  13.50




Anova is robust to a small number of outliers.

One-way ANOVA results:
F-statistic: 4.7107
p-value: 0.001438
Alpha: 0.05

REJECT H0 (p < 0.05)
Interest rates DO differ significantly between cities.



Well, most analyses aren't so simple. Let's have a look at another dataset. This one is located [here](http://www.itl.nist.gov/div898/education/anova/ratfeed.dat). Get acquainted with it.

This time, we've got two explanatory variables: amount and type of diet.

Perform one-way ANOVA to each variable separately, disregarding the other one. 

To do this, take for example the "Amount" variable. Group all weights by amount, disregarding the diet type. Perform ANOVA on the groups, like you did before. Repeat the same process with "Diet".

Do the groups differ significantly at the 5%-level?

In [ ]:
# Write your code here

Sometimes we have to take into account multiple values. In this case, we want to ask another question: Is there a significant difference between combinations of diet and amount? These are called **interacting variables**. Unfortunately, there isn't a quick and easy way to perform this test in `scipy.stats`, but `statsmodels` is another library which will come in handy.

We won't go into many details how it works but you basically create a linear model (`ols` stands for "ordinary least squares") and you provide a formula. The formula we want to use looks like this:
```python
formula = "Weight ~ C(Amount) + C(Diet) + C(Amount):C(Diet)"
```

`Weight`, `Amount` and `Diet` are the **column names** (i.e. variable names) in the dataset. `C(Amount)` tells the library to treat the variable as *categorical*, not numerical (e.g. diet 1 + diet 3 makes absolutely no sense). Finally `C(Amount):C(Diet)` means that we take the interaction between these categories (Cartesian product - every amount with every diet).

How do we read this formula? "The Weight column is proportional to the categories Amount, Diet and their interaction" (i.e. Weight is a linear combination of the three other variables).

We are now ready. Well, we could have done all this by hand as well, but why bother :)? Just create the linear model:
```python
model = ols(formula, rats).fit()
```

and the ANOVA table:
```python
table = statsmodels.stats.anova.anova_lm(model, typ = 2)
```

You can see there's "type 2" ANOVA chosen. If you're interested, you can read more about the three types of ANOVA [here](https://mcfromnz.wordpress.com/2011/03/02/anova-type-iiiiii-ss-explained/)

P. S.: The type of formulas we described above are typical for another popular language for statistics and data analysis, called **R**. They're also why many people hate R. They're good but need a bit of understanding and getting accustomed to. Case closed :D.

In [ ]:
# Write your code here

### Problem 3. Contingency Tables. $\chi^2$-test
Have a look at the dataset `data/Popular Kids.tsv` and its description `data/Popular Kids Description.txt`.

When we want to compare data across many categories, a useful way we can visualize it, is using **contingency tables** (or two-way tables). One variable goes to the rows, another one - to the columns of the table. The intersection cell represents the number of observations having this combination of values.

Try it now:
```python
pd.crosstab(kids.Grade, kids.Goals)
```

You can also specify multiple columns - this will create higher-dimensional tables. Keep in mind, however, that these are difficult to look at and understand. We're usually fine with two variables, three in some rare cases.

In [ ]:
# Write your code here

The **$\chi^2$**-test provides a way to test for association between two (or more) categorical variables. In this case, **Grade** and **Goals** seem to be good candidates. This is most easily done using the contingency table. Fortunately, `scipy.stats` has one such method (read the docs [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html)).

Note that this test doesn't specify what the relationship is. It just tells us that there might be a significant relationship.

Are goals related to grade? To what extent? Are these significant?
* $H_0$: No relation between Grade and Goals
* $H_1$: Some relation exists between Grade and Goals
* $\alpha = 0,05$

In [ ]:
# Write your code here

The school areas are also divided into "Urban", "Suburban" and "Rural". Are the school areas related to goals? Perform the same type of test. Is the difference significant now?

In [ ]:
# Write your code here

Feel free to continue exploring the dataset as you wish. You can find interesting data and relationships. 

**Note:** You'll see the catch of categorical variables - these are very difficult to visualize properly. Most plots just appear as symmetrical patterns of dots. You can rely on contingency tables and correlations to properly describe and explore these variables.

### Problem 4. Image Convolution
"Convolution" refers to a process similar to "mixing" two functions. It can be performed in one dimension (e.g. on audio data), or many dimensions. In this problem, we'll look at 2D convolution of images and what we can do with it.

Let's first read an image. Once again, to make things easier, we'll separate the channels. We can work on all three channels separately or at once but it's easier to work with one channel only.

In [ ]:
original_image = imread("https://upload.wikimedia.org/wikipedia/commons/d/d9/Norwegian_Forest_Cat_Portrait.JPG")

def display(image):
    # If there is only one channel to show, display it as grayscale
    cm = None
    if(len(image.shape)) == 2:
        cm = "gray"
    plt.figure(figsize = (5, 10))
    plt.imshow(image, cmap = cm)
    plt.xticks([])
    plt.yticks([])
    plt.show()

display(original_image)
r = original_image[:, :, 0]
display(r)

**Convolution** means taking a special square matrix (usually 3x3 or 5x5), called a **convolution kernel** and applying it to the image like this: the central pixel of the resulting image is the sum of element-wise products between the image and the kernel:
![Convolution example](convolution.png)

After that, the kernel moves 1px to the right and contiinues. It "slides" across the entire image. The edge pixels are a bit problematic but there are several ways to deal with that. The most common way is to copy whatever pixel value happened to be at the border.

![Border handling](border.png)

The algorithm is always the same. The output depends on the **kernel**. Different kernels produce different results: some detect edges (lines), others detect corners; some apply blurring and sharpening; some remove noise, etc.

The results can be useful for analyzing what's on the image, or just for artistic purposes.

Let's examine this filter, for example:

$$ F = \begin{bmatrix}
1/9 & 1/9 & 1/9 \\
1/9 & 1/9 & 1/9 \\
1/9 & 1/9 & 1/9
\end{bmatrix}
$$

This calculates the average of all surrounding pixels and basically smooths the image.

Note that in order to preserve brightness, the sum of all elements in $F$ must be equal to 1. If it's not, the image will be darker or brighter (which may or may not be a desired effect).

`scipy.ndimage` has a method for performing 1D and multi-dimensional convolution. Read the docs [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.ndimage.filters.convolve.html#scipy.ndimage.filters.convolve).

Apply the convolution. To see better how it performs, you can plot only a part of the image - this will zoom the entire thing. Compare the "before" and "after" images.

In [ ]:
# Write your code here

Play around with more kernels (they're also called **filters**). You can find examples on the Internet, or you can create your own. Have fun :).

Try these filters:
* Gaussian blur
* Sobel edge detector - vertical, horizontal
* Corner detector
* Gradient detector
* Sharpening
* Unsharp mask

For each filter, show the result before and after its application.

Sources: <a href="https://en.wikipedia.org/wiki/Kernel_(image_processing)">Wikipedia</a>, [Image Kernels explained visually](http://setosa.io/ev/image-kernels/).

**Optional:** Think about you might use edge, corner and gradient detectors in image processing.

In [ ]:
# Write your code here

### ** Problem 5. Classification
A type of widely used **models** is **classification**. Regression outputs a continuous value while classification outputs one of several pre-defined classes. In the most simple way, the classes are only two. For example, if we want to detect whether there's a cat on an image, we can have two classes: "cat" and "non-cat".

Explore the problem of classification. Implement and document one algorithm. Apply it to some real-world data. You can use the following checklist:

**Note:** If your paper is **about the algorithm**, consider **writing it from scratch**, not reusing it from a library.

* What is supervised learning? What do supervised learning models do?
* What is regression? What is classification?
* What types of problems does classification solve directly?
    * What types of problems can be reduced to classification?
* What's the difference between two-class and multi-class classification?
* Explore one algorithm for classification, e.g. logistic regression.
    * State the problem clearly
    * List all sssumptions of the modelling function
    * Describe the process: distances, error function, total loss, gradient descent, etc.; as needed
    * Implement the algorithm from scratch
* Select or generate a small dataset, suitable for classification. Run your algorithm as a sanity check
* Debug and solve any problems
* Waht is a confusion matrix?
* What metrics are used to score a classifier?
    * Accuracy, Precision, Recall, others
    * ROC curve, interpretation
* Select a real dataset
    * Explore it to get acquainted with what information it contains
    * Clean up the data if you need to do so
    * Perform classification
    * Score your classification model
    * Use your classifier to predict
        * Split the data into training and testing set
        * Optionally, perform **cross-validation**
    * Compare your implementation to another one, e.g. `scikit-learn`. They should give the same (or very similar) results
    * Communicate the results on your dataset
    * Optionally, publish your model on the Internet as a Web API

### ** Problem 6. Fourier Transform Applications: Images
Examine how Fourier transform is used in image processing. Consider writing the algorithms for transformation (forward and inverse) from scratch.

You may look at other transformations as well, for example, the **cosine transform** (used in JPEG images to achieve compression) or **wavelet transform**. You can use the following checklist:

* What is Fourier transformation? What information does it provide?
    * What do the forward and inverse transformation do? What are their inputs and outputs?
    * How do we interpret the results ("time domain" / "frequency domain")?
        * What does this mean in 2D? How does a 1D spectrum differ from a 2D spectrum?
        * What are the characteristic parts of a 2D spectrum?
    * Give some examples of well-known functions and their transformations
* What is Discrete Fourier Transform (DFT)?
    * Implement the algorithm in 2D
* How do we use DFT in image processing?
    * Provide examples
* Select an image and perform operations on it using your implementation of DFT
    * Low-pass filtering
    * High-pass filtering
    * Finding and removal of periodic component from an image
    * Fnding and removal of peaks
    * Shape analysis
* How do convolutions relate to DFT?
    
[This document](http://www.robots.ox.ac.uk/~az/lectures/ia/lect2.pdf), [this document](http://research.stowers.org/mcm/efg/Report/FourierAnalysis.pdf), and many others provide a good starting point. You can find a lot of examples in digital image processing and art using Fourier transformations.